In [1]:
import json
import codecs
import glob
import os

In [2]:
rasaStoriesFile = 'converted_files/data/stories.md'
rasaUtterancesFile = 'converted_files/data/nlu.md'
rasaDomainFile = 'converted_files/domain.yml'

In [3]:
intents = {}
all_slots = set()
all_responses = set()
stories = {}

# Stories

In [5]:
def breakdown_input(convo):
    val = convo['semantics']['cam']
    val = val.replace('(', '{"')
    val = val.replace(')', '"}')
    val = val.replace('=', '": "')
    val = val.replace(',', '", "')
    val = val.replace('{""}', '')
    val = val.replace('""', '"')
    return (val)

In [6]:
def remove_or(somestring):
    if "|bye" in somestring: 
        return "bye"
    return somestring

In [7]:
def set_request_param(val):
    val = val.replace('{', '{"slot": ')
    val = val.replace(',', ', "slot": ')
    return val

In [8]:
def remove_action_conflict(story):
    indices = set()
    for i in range(0, len(story) - 1):
        if story[i] ==' - action_request' and story[i+1] ==' - action_suggest':
            indices.add(i+1)
    return [v for i, v in enumerate(story) if i not in indices]

In [9]:
# stories

suggests = ['offer','select', 'canthelp']
requests = 'request'
def get_story(log, label):
    # print('## story_' + log['session-id'])
    story = '## story_' + log['session-id']
    li = ['* hello']
    for i in range(0, len(log['turns'])):
        out = log['turns'][i]['output']['dialog-acts'][0]['act']
        
        if out in suggests:
            li.append(' - action_suggest')
            
        li.append(' - utter_' + out)
        all_responses.add(' utter_'+out)
        t = [a for a in label['turns'] if a['turn-index'] == i]
        inpt = remove_or(breakdown_input(t[0]))
        
        if requests in inpt:
            inpt = set_request_param(inpt)
        
        s = '* ' + inpt
        li.append(s)
        
        if requests in inpt:
            li.append(' - action_request')
    
    li.append(' - utter_')    
    li = remove_action_conflict(li)
    stories[story] = li

# NLU

In [10]:
# NLU
def get_nlu(log, label):
    for i in range(0, len(label['turns'])):
        turn = [a for a in label['turns'] if a['turn-index'] == i][0]

        usr_input = turn['transcription']
        slots = turn['semantics']['json']
        
        if slots:
            
            slots = slots[0]['slots']
            intent = turn['semantics']['json'][0]['act']

            if intent not in intents:
                intents[intent] = set()

            if slots:
                for slot in slots:
                    all_slots.add(slot[0])
                    spl = usr_input.split()
                    index = [i for i in range(len(spl)) if spl[i] == slot[1]]
                    for ind in index:
                        spl[ind] = '['+spl[ind]+']'+'('+ slot[0]+')'
                    usr_input = " ".join(spl)

            intents[intent].add(usr_input)       


# Apply To All

In [11]:
[x[0] for x in os.walk('./data')][1:]

['./data/voip-2d3d74d091-20130328_135311',
 './data/voip-03c2655d43-20130327_194616',
 './data/voip-fe4b6ef58f-20130325_220934',
 './data/voip-d225fad9df-20130328_183352',
 './data/voip-b57f8ee22b-20130327_000138',
 './data/voip-59bc8a2167-20130325_143706',
 './data/voip-0fa32b1e78-20130328_151336',
 './data/voip-50af5438f1-20130402_084400',
 './data/voip-a31ca3e355-20130323_223338',
 './data/voip-e8997b10da-20130401_152019',
 './data/voip-4f069a4136-20130402_031309',
 './data/voip-22756d9e8f-20130329_045435',
 './data/voip-ab4f1dbb59-20130328_180542',
 './data/voip-4660dd9eab-20130329_080055',
 './data/voip-4a6ecc1f1c-20130328_121528',
 './data/voip-8d5173f3a6-20130324_184603',
 './data/voip-597cfafdee-20130328_234346',
 './data/voip-22756d9e8f-20130329_050114',
 './data/voip-317a1436fe-20130325_172154',
 './data/voip-e9b53d6ace-20130401_205843',
 './data/voip-597cfafdee-20130402_010910',
 './data/voip-b6618de447-20130325_145518',
 './data/voip-0f41c16f2f-20130401_235017',
 './data/vo

In [12]:
directories = [x[0] for x in os.walk('./data')][1:]

for direct in directories:
    print(direct)
    with open(direct+'/log.json') as json_file:
        main_log = json.load(json_file)

    with open(direct+'/label.json') as json_file:
        main_label = json.load(json_file)
    
    get_story(main_log, main_label)
    get_nlu(main_log, main_label)

./data/voip-2d3d74d091-20130328_135311
./data/voip-03c2655d43-20130327_194616
./data/voip-fe4b6ef58f-20130325_220934
./data/voip-d225fad9df-20130328_183352
./data/voip-b57f8ee22b-20130327_000138
./data/voip-59bc8a2167-20130325_143706
./data/voip-0fa32b1e78-20130328_151336
./data/voip-50af5438f1-20130402_084400
./data/voip-a31ca3e355-20130323_223338
./data/voip-e8997b10da-20130401_152019
./data/voip-4f069a4136-20130402_031309
./data/voip-22756d9e8f-20130329_045435
./data/voip-ab4f1dbb59-20130328_180542
./data/voip-4660dd9eab-20130329_080055
./data/voip-4a6ecc1f1c-20130328_121528
./data/voip-8d5173f3a6-20130324_184603
./data/voip-597cfafdee-20130328_234346
./data/voip-22756d9e8f-20130329_050114
./data/voip-317a1436fe-20130325_172154
./data/voip-e9b53d6ace-20130401_205843
./data/voip-597cfafdee-20130402_010910
./data/voip-b6618de447-20130325_145518
./data/voip-0f41c16f2f-20130401_235017
./data/voip-31d9d1a567-20130402_035757
./data/voip-e0035cc31b-20130326_210405
./data/voip-bde2721237-20

./data/voip-10beae627f-20130328_171313
./data/voip-be5b7bf9d9-20130401_152435
./data/voip-bb1fd497eb-20130325_163635
./data/voip-7e07d8f0f5-20130327_174435
./data/voip-48c12815b3-20130326_011113
./data/voip-597cfafdee-20130328_232032
./data/voip-be5b7bf9d9-20130402_202555
./data/voip-52d599db9c-20130326_010952
./data/voip-e3b4879e0d-20130326_021022
./data/voip-869dd52548-20130401_180331
./data/voip-3b3edac94d-20130324_201732
./data/voip-7f9c1c8411-20130328_200359
./data/voip-7e07d8f0f5-20130328_184956
./data/voip-78f497f314-20130324_140601
./data/voip-869dd52548-20130401_184559
./data/voip-199d62165b-20130402_123401
./data/voip-4c25da9a27-20130325_183445
./data/voip-59bc8a2167-20130328_130810
./data/voip-d645d56d23-20130323_222004
./data/voip-14f776f781-20130329_033249
./data/voip-58047f5227-20130327_034311
./data/voip-be5694f464-20130328_125916
./data/voip-3b59a0391b-20130401_134901
./data/voip-dd9f7810fd-20130322_225458
./data/voip-fce37b0ccb-20130328_145014
./data/voip-52eb280e7b-20

./data/voip-5749b16764-20130328_150400
./data/voip-e8997b10da-20130327_194306
./data/voip-381a50592b-20130323_233439
./data/voip-dd9f7810fd-20130322_232152
./data/voip-fe2783c40a-20130401_143116
./data/voip-3b81cbb287-20130324_015234
./data/voip-50af5438f1-20130402_085647
./data/voip-0f41c16f2f-20130401_235748
./data/voip-2d2d103292-20130326_041008
./data/voip-d76851034e-20130326_221320
./data/voip-e0035cc31b-20130323_212626
./data/voip-4660dd9eab-20130329_085648
./data/voip-381a50592b-20130326_041932
./data/voip-3b81cbb287-20130326_031714
./data/voip-e9b53d6ace-20130324_222525
./data/voip-d645d56d23-20130401_203325
./data/voip-90732b027d-20130401_222034
./data/voip-7e07d8f0f5-20130328_185739
./data/voip-5cf59cc660-20130327_144338
./data/voip-7f9c1c8411-20130328_210227
./data/voip-908884f5fd-20130327_000655
./data/voip-a352cb5ca5-20130401_231236
./data/voip-da10d74c3e-20130326_001657
./data/voip-4c0d36762a-20130328_204534
./data/voip-aaa44b4121-20130327_170541
./data/voip-935947e17b-20

# Write

In [13]:
open(rasaUtterancesFile, 'w').close()
open(rasaStoriesFile, 'w').close()
open(rasaDomainFile, 'w').close()

In [14]:
def write_nlu(writeFile, intents):
    with codecs.open(writeFile, 'w', 'utf-8') as f_out:
        for k, vals in intents.items():
            f_out.write('## intent:%s\n' %k.lower())
            for val in vals:
                f_out.write('- %s\n' %val)
            f_out.write('\n')

In [15]:
write_nlu(rasaUtterancesFile, intents)

In [16]:
def write_stories(writeFile, stories):
    with codecs.open(writeFile, 'w', 'utf-8') as f_out:
        for k, vals in stories.items():
            f_out.write('%s\n' %k.lower())
            for val in vals:
                f_out.write('%s\n' %val)
            f_out.write('\n')

In [17]:
write_stories(rasaStoriesFile, stories)

In [18]:
def write_domain(writeFile, intents, stories, slots, responses):
    with codecs.open(writeFile, 'w', 'utf-8') as f_out:
        f_out.write('slots:\n')
        for slot in slots:
            f_out.write(' %s:\n' %slot.lower())
            f_out.write('  type : text \n')
        f_out.write('\nentities:\n')
        for slot in slots:
            f_out.write('- %s\n' %slot.lower())
        f_out.write('\nintents:\n')
        for k in intents:
            f_out.write(' - %s\n' %k.lower())
        f_out.write('\nresponses:\n')
        for resp in responses:
            f_out.write(' %s:\n' %resp.lower())
            f_out.write('  - text: YOUR CHOSEN TEXT \n')
            
        f_out.write('\nactions:\n')
        f_out.write('- action_suggest\n')
        f_out.write('- action_request')

In [19]:
write_domain(rasaDomainFile, intents, stories, all_slots, all_responses)